# Práctica 1 - Self-Organising Maps
## Preparación de entorno
#### Importar librerías de código

In [141]:
%reset
# from __future__ import division

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as patches

%matplotlib inline

Nothing done.


#### Dataset que se va a utilizar para el entrenamiento

In [142]:
# Código para obtener el Dataset que se va a usar en el entrenamiento
valor_min = 0
valor_max = 256     #256 porque el valor máximo es exclusivo, es decir, irá de 0 a 255
valores_color = 3
num_colores = 100
datos = np.random.randint(valor_min, valor_max, (num_colores, valores_color))
print(f"La matriz de 100 colores:\n",datos)







La matriz de 100 colores:
 [[163  56 221]
 [ 24 255 132]
 [193 154 250]
 [ 14 216  43]
 [199 109 216]
 [240 213 248]
 [ 49 105 219]
 [ 96 192 215]
 [164  78 208]
 [135 174  80]
 [167 241  93]
 [ 66  39 117]
 [ 86 175 204]
 [231 221 242]
 [239 120 155]
 [  3  19 134]
 [ 50 213 159]
 [222  86 238]
 [ 52 211 235]
 [  6  26 218]
 [149 228  56]
 [204 100  98]
 [ 80 178 185]
 [ 97  11 253]
 [ 98 142 241]
 [123  96  87]
 [222  72  91]
 [108 121 229]
 [225 241 215]
 [250 132   6]
 [ 37  89 129]
 [255  50 118]
 [ 34 124 153]
 [183  40 198]
 [212 216 230]
 [170 249  95]
 [252  95  66]
 [175  99 110]
 [156 232 206]
 [ 32 170 100]
 [100 253   7]
 [176 175  61]
 [111 149 141]
 [208  23 223]
 [ 16 167   2]
 [ 19 214 117]
 [ 48 149  28]
 [ 28 160 169]
 [224   1  83]
 [ 46 225 115]
 [227  34  34]
 [183  66 157]
 [159  27 230]
 [ 12  34 127]
 [194 133  83]
 [ 37  38  33]
 [206  88 169]
 [  5  18  76]
 [245   3  39]
 [128 132  18]
 [233  33 159]
 [ 33 243 231]
 [133 115 118]
 [ 84 209  79]
 [ 34  25 234

## SOM Setup
#### Variables definidas por el alumno

In [143]:
# Inicializa tamaño del mapa de Kohonen, número de iteraciones y learning rate
# Inicializa normalizar_datos dependiendo de si tienes que normalizar los datos o no
lado_mapa = 10
periodo = 1000
learning_rate = 0.1
normalizar_datos = True

#### A partir de este punto solo hay cálculos. No se introducen más valores "a mano"

In [144]:
# Establece el numero de entradas del mapa y el número de datos que se van a usar para entrenar. 
# Utiliza una función que obtenga automáticamente los valores a partir del Dataset.

num_entradas = len(datos[0])   #Usamos len para contar el numero de filas y columnas
num_datos = len(datos)     

def obtener_Entrada(datos):
   valor_aleatorio = np.random.randint(0,101)
   return datos[valor_aleatorio]

# Calcula el vecindario inicial. Debe ser la mitad del lado del mapa de Kohonen
vecindario = lado_mapa/2

# Normaliza los datos si fuese necesario dividiendo cada dato por el máximo en la matriz
# Calculamos el valor maximo de la matriz

valor_maximo = 0
for dato in datos:
    for valor in dato:
        if(valor_maximo < valor):
            valor_maximo = valor  
      
if normalizar_datos:
    datos = datos/valor_max    
    
    
# Crea una matriz de pesos con valores random entre 0 y 1. Usa la función random.random de la librería NumPy
#matriz_pesos = np.random.random((dimensiones de la matriz de pesos))

matriz_pesos = np.random.random((lado_mapa*lado_mapa,num_entradas))
print(matriz_pesos)

[[0.86699513 0.68383466 0.19471528]
 [0.47021848 0.74957853 0.4712039 ]
 [0.1238536  0.1258544  0.09451756]
 [0.80458941 0.88196239 0.48575254]
 [0.55047235 0.05838932 0.54317839]
 [0.71508637 0.66226373 0.61873571]
 [0.15207757 0.08276381 0.64211744]
 [0.4568903  0.27361821 0.43568293]
 [0.37679782 0.7927875  0.59769316]
 [0.1076786  0.11797743 0.78048186]
 [0.89473062 0.26053691 0.39267105]
 [0.32359995 0.74847136 0.74484004]
 [0.687746   0.04240668 0.17363943]
 [0.71459806 0.36229629 0.45465677]
 [0.34605775 0.87373056 0.41804182]
 [0.60097453 0.15082266 0.86058388]
 [0.16714996 0.52043544 0.65399528]
 [0.15006007 0.61887753 0.124609  ]
 [0.3602151  0.78567838 0.67131736]
 [0.82406101 0.65983926 0.46697843]
 [0.50765085 0.99649696 0.09365209]
 [0.36569845 0.39651522 0.71640662]
 [0.69361662 0.54709892 0.19809803]
 [0.36803691 0.90378242 0.61539608]
 [0.75859822 0.85009347 0.4941851 ]
 [0.99307419 0.92414259 0.58739743]
 [0.21140358 0.9053684  0.347782  ]
 [0.78151317 0.13914457 0.29

#### Funciones para entrenar/clasificar

In [ ]:
# Función para encontrar la BMU
"""
   Encuentra la BMU para un patrón de entrada.
   Entradas: (patrón_de_entrada, matriz_de_pesos, número_de_entradas)
   Salidas:  (bmu, bmu_idx) tupla donde
               bmu: vector de pesos de la neurona ganadora
               bum_idx: coordenadas de la neurona ganadora
"""

def calcular_bmu(patron_entrada, matriz_pesos, m):
   
   """
    Diferencias es un array de 3 elementos que tendrá el cuadrado de la resta de la entrada menos el peso para calcular la distancia euclídea que
    terminará calculándose en distancia local haciendo la raíz de la suma de los valores anteriores.
   
   """
   distancia_local = 0                      
   distancia_minima = float('inf')
   diferencias = np.zeros(m)
   bmu = []
   bmu_idx = ()

   for i in range(len(matriz_pesos)):
      for j in range(m):
         diferencias[j] = (patron_entrada[j] - matriz_pesos[i,j]) ** 2
      """

      Distancia local tendrá la distancia que se esté calculando en el momento, es decir, en la primera iteración tendrá la distancia de la 
      neurona 0,0 al patrón de entrada, después la de 0,1 y así sucesivamente.
      Si la distancia local es menor que la distancia mínima consideraremos esa neurona como la bmu. En la primera iteración la primera neurona 
      será la bmu ya que distancia mínima está inicializada a infinito.
      
      """
      distancia_local = np.sqrt(np.sum(diferencias))
      if(distancia_local < distancia_minima):
         distancia_minima = distancia_local
         bmu = matriz_pesos[i]
         bmu_idx = (i//10,i%10)

   # Por último retornamos la bmu y su posición
   return (bmu, bmu_idx)
           
patron_entrada = obtener_Entrada(datos)      
bmu = calcular_bmu(patron_entrada, matriz_pesos, num_entradas)
print(f"El patrón de entrada es:\n",patron_entrada)
print(f"Y la BMU es:")
print(f"Valor:",bmu[0],"\nPosicion:",bmu[1])

    
    

El patrón de entrada es:
 [0.28125    0.6875     0.23828125]
Y la BMU es:
Valor: [0.42357245 0.79053664 0.29429338] 
Posicion: (8, 8)


In [ ]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta)
"""
   Calcula el Learning Rate (eta) que corresponde a la i-ésima presentación.
   Entradas: (learning_rate_inicial, iteracion, período)
   Salidas:  learning_rate para la iteración i

"""
#Aplicamos la función para calcular la variación de lr
def variacion_learning_rate(lr_inicial, i, n_iteraciones):
    return lr_inicial * (1-(i/n_iteraciones)) 

SyntaxError: incomplete input (3295362566.py, line 8)

In [ ]:
# Función para calcular el descenso del vecindario (v)
"""
   Calcula el vecindario  (v) que corresponde a la i-ésima presentación.
   Entradas: (vecindario_inicial, iteracion, período)
   Salidas:  lvecindario para la iteración i

"""
def variacion_vecindario(vecindario_inicial, i, n_iteraciones):
   return 1 + vecindario_inicial * (1-(i/n_iteraciones))

In [ ]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta) en función de la distancia a la BMU
"""
   Calcula la amortiguación de eta en función de la distancia en el mapa entre una neurona y la BMU.
   Entradas: (distancia_BMU, vecindario_actual)
   Salidas:  amortiguación para la iteración

"""
def decay(distancia_BMU, vecindario_actual):
    return np.exp(-distancia_BMU / (2* (vecindario_actual**2)))

#### Funciones para dibujar la salida de la red

In [ ]:
# Función para pintar una matriz de valores como colores RGB
def pintar_mapa(matriz_valores):
    fig = plt.figure()
    
    # Establece ejes
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim((0, matriz_pesos.shape[0]+1))
    ax.set_ylim((0, matriz_pesos.shape[1]+1))
    ax.set_title('Self-Organising Map después de %d iteraciones' % periodo)

    # Dibuja los rectángulos de color RGB
    for x in range(1, matriz_valores.shape[0] + 1):
        for y in range(1, matriz_valores.shape[1] + 1):
            ax.add_patch(patches.Rectangle((x-0.5, y-0.5), 1, 1,
                         facecolor=matriz_valores[x-1,y-1,:],
                         edgecolor='none'))
    plt.show()

## SOM Entrenamiento

In [ ]:
# Entrena la red con el dataset de entrenamiento



## SOM Clasificación

In [ ]:
# Clasifica los patrones de entrenamiento con la matriz de pesos recién entrenada



## SOM Prueba

In [ ]:
# Clasifica nuevos patrones

